In [9]:
import torchaudio
from speechbrain.pretrained import EncoderClassifier
language_id = EncoderClassifier.from_hparams(source="speechbrain/lang-id-voxlingua107-ecapa", savedir="tmp")

In [105]:
import os 
import pandas as pd

def load_data(root_dir):
    files = {}
    # if for .DS_Store file, thanks mac
    for lang_dir in [fname for fname in os.listdir(root_dir) if len(fname) == 2]:
        lang_samples = [f'{root_dir}/{lang_dir}/{fname}' for fname in os.listdir(f'{root_dir}/{lang_dir}')]
        files[f"{root_dir}/{lang_dir}"] = lang_samples
        
    return files

def get_embeddings(embedding_dict):
    emb_df = pd.DataFrame(columns=[x for x in range(256)])
    meta = []
    for i, (key, val) in enumerate(embedding_dict.items()):
        for j, wav_name in enumerate(val):
            signal = language_id.load_audio(wav_name, savedir="tmp_save")
            emb_df = pd.concat([emb_df, pd.DataFrame(np.array(language_id.encode_batch(signal))[0][0].T).T])
            meta.append(key.split('/')[-1])
        
            print("{}/{}: {:.2%}".format(i, len(embedding_dict), j/len(val)))
            
    return pd.Series(meta), emb_df

In [106]:
meta, emb_df = get_embeddings(load_data('dev'))
emb_df.to_csv('emb_test.tsv', sep='\t', header=None, index=False)
meta.to_csv('emb_meta_test.tsv', sep='\t', header=None, index=False)

0/33: 0.00%
0/33: 11.11%
0/33: 22.22%
0/33: 33.33%
0/33: 44.44%
0/33: 55.56%
0/33: 66.67%
0/33: 77.78%
0/33: 88.89%
1/33: 0.00%
1/33: 1.92%
1/33: 3.85%
1/33: 5.77%
1/33: 7.69%
1/33: 9.62%
1/33: 11.54%
1/33: 13.46%
1/33: 15.38%
1/33: 17.31%
1/33: 19.23%
1/33: 21.15%
1/33: 23.08%
1/33: 25.00%
1/33: 26.92%
1/33: 28.85%
1/33: 30.77%
1/33: 32.69%
1/33: 34.62%
1/33: 36.54%
1/33: 38.46%
1/33: 40.38%
1/33: 42.31%
1/33: 44.23%
1/33: 46.15%
1/33: 48.08%
1/33: 50.00%
1/33: 51.92%
1/33: 53.85%
1/33: 55.77%
1/33: 57.69%
1/33: 59.62%
1/33: 61.54%
1/33: 63.46%
1/33: 65.38%
1/33: 67.31%
1/33: 69.23%
1/33: 71.15%
1/33: 73.08%
1/33: 75.00%
1/33: 76.92%
1/33: 78.85%
1/33: 80.77%
1/33: 82.69%
1/33: 84.62%
1/33: 86.54%
1/33: 88.46%
1/33: 90.38%
1/33: 92.31%
1/33: 94.23%
1/33: 96.15%
1/33: 98.08%
2/33: 0.00%
2/33: 50.00%
3/33: 0.00%
3/33: 1.00%
3/33: 2.00%
3/33: 3.00%
3/33: 4.00%
3/33: 5.00%
3/33: 6.00%
3/33: 7.00%
3/33: 8.00%
3/33: 9.00%
3/33: 10.00%
3/33: 11.00%
3/33: 12.00%
3/33: 13.00%
3/33: 14.00%
3/33

13/33: 55.17%
13/33: 58.62%
13/33: 62.07%
13/33: 65.52%
13/33: 68.97%
13/33: 72.41%
13/33: 75.86%
13/33: 79.31%
13/33: 82.76%
13/33: 86.21%
13/33: 89.66%
13/33: 93.10%
13/33: 96.55%
14/33: 0.00%
15/33: 0.00%
15/33: 4.35%
15/33: 8.70%
15/33: 13.04%
15/33: 17.39%
15/33: 21.74%
15/33: 26.09%
15/33: 30.43%
15/33: 34.78%
15/33: 39.13%
15/33: 43.48%
15/33: 47.83%
15/33: 52.17%
15/33: 56.52%
15/33: 60.87%
15/33: 65.22%
15/33: 69.57%
15/33: 73.91%
15/33: 78.26%
15/33: 82.61%
15/33: 86.96%
15/33: 91.30%
15/33: 95.65%
16/33: 0.00%
16/33: 5.88%
16/33: 11.76%
16/33: 17.65%
16/33: 23.53%
16/33: 29.41%
16/33: 35.29%
16/33: 41.18%
16/33: 47.06%
16/33: 52.94%
16/33: 58.82%
16/33: 64.71%
16/33: 70.59%
16/33: 76.47%
16/33: 82.35%
16/33: 88.24%
16/33: 94.12%
17/33: 0.00%
17/33: 14.29%
17/33: 28.57%
17/33: 42.86%
17/33: 57.14%
17/33: 71.43%
17/33: 85.71%
18/33: 0.00%
18/33: 25.00%
18/33: 50.00%
18/33: 75.00%
19/33: 0.00%
19/33: 1.00%
19/33: 2.00%
19/33: 3.00%
19/33: 4.00%
19/33: 5.00%
19/33: 6.00%
19/33: 

27/33: 16.36%
27/33: 18.18%
27/33: 20.00%
27/33: 21.82%
27/33: 23.64%
27/33: 25.45%
27/33: 27.27%
27/33: 29.09%
27/33: 30.91%
27/33: 32.73%
27/33: 34.55%
27/33: 36.36%
27/33: 38.18%
27/33: 40.00%
27/33: 41.82%
27/33: 43.64%
27/33: 45.45%
27/33: 47.27%
27/33: 49.09%
27/33: 50.91%
27/33: 52.73%
27/33: 54.55%
27/33: 56.36%
27/33: 58.18%
27/33: 60.00%
27/33: 61.82%
27/33: 63.64%
27/33: 65.45%
27/33: 67.27%
27/33: 69.09%
27/33: 70.91%
27/33: 72.73%
27/33: 74.55%
27/33: 76.36%
27/33: 78.18%
27/33: 80.00%
27/33: 81.82%
27/33: 83.64%
27/33: 85.45%
27/33: 87.27%
27/33: 89.09%
27/33: 90.91%
27/33: 92.73%
27/33: 94.55%
27/33: 96.36%
27/33: 98.18%
28/33: 0.00%
28/33: 1.18%
28/33: 2.35%
28/33: 3.53%
28/33: 4.71%
28/33: 5.88%
28/33: 7.06%
28/33: 8.24%
28/33: 9.41%
28/33: 10.59%
28/33: 11.76%
28/33: 12.94%
28/33: 14.12%
28/33: 15.29%
28/33: 16.47%
28/33: 17.65%
28/33: 18.82%
28/33: 20.00%
28/33: 21.18%
28/33: 22.35%
28/33: 23.53%
28/33: 24.71%
28/33: 25.88%
28/33: 27.06%
28/33: 28.24%
28/33: 29.41%
2

In [ ]:
# translate meta file
